In [ ]:
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.session import Session
from sagemaker.tuner import (
    ContinuousParameter,
    HyperparameterTuner,
)

bucket = "BUCKET_NAME"
project_path = "jonas/abalone"
output_path = f"s3://{bucket}/{project_path}/output"

sagemaker_session = Session()
train = sagemaker_session.upload_data("local/data", bucket=bucket, key_prefix='jonas/abalone/train', extra_args=None)
source_dir = sagemaker_session.upload_data("script", bucket=bucket, key_prefix='jonas/abalone/script', extra_args=None)

K = 5
hyperparams = {
    "eta": 0.1,
    "num_class": 30,
    "num_round": 20,
    "objective": "multi:softprob",
    "hite_rate_k": K
}

estimator = XGBoost(
    entry_point="train.py",
    source_dir=source_dir,
    hyperparameters=hyperparams,
    role="AML-SageMakerRole",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version="1.3-1",
    output_path=output_path,
    sagemaker_session=sagemaker_session
)


tuner = HyperparameterTuner(
    estimator=estimator,
    objective_metric_name=f'validation:hit_rate_at_{K}',
    hyperparameter_ranges={
        "eta": ContinuousParameter(0, 0.3)
    },
    metric_definitions=[
        {'Name': f'validation:hit_rate_at_{K}', 'Regex': '(validation-hit_rate_at_\d):(\d+(?:\.\d+)?)'}
    ],
    max_jobs=2,
    max_parallel_jobs=2
)

tuner.fit(
    inputs = {
        "train": train,
        "validation": train
    }
)
